<a href="https://colab.research.google.com/github/AmanullaChishty/FoodClassifier/blob/main/Food_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Food Classifier built on EffNetB2 model